"""Which Classifier is Should I Choose?
This is one of the most import questions to ask when approaching a machine learning
problem.I find it easier to just test them all at once. """

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
import sys


print ('Total Number of arguments:', len(sys.argv), 'arguments.')
print ('Argument List:', str(sys.argv))
print (sys.argv[0])


In [ ]:
#Data Preparation

In [ ]:
# Swiss army knife function to organize the data
def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species) # encode species strings
    classes = list(le.classes_)       # save column names for submission
    test_ids = test.id                # save test ids for submission
    
    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)
train.shape


In [ ]:
train.head()

In [ ]:
len(labels)

In [ ]:
labels

In [ ]:
test.head()

In [ ]:
len(test_ids)

In [ ]:
test_ids

In [ ]:
classes

In [ ]:
len(classes)

In [ ]:
print(labels.shape)
print(test.shape)
print(test_ids.shape)
print(classes)


"""Stratified Train/Test Split - Stratification is necessary for this dataset because
there is a relatively large number of classes (100 classes for 990 samples). This 
will ensure we have all classes represented in both the train and test indices"""

In [ ]:
sss = StratifiedShuffleSplit( n_splits=10, test_size=0.3, random_state=23)
print(sss.get_n_splits(train,labels))
for train_index, test_index in sss.split(train,labels):
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

"""Sklearn Classifiers
Simply looping through 4 classifiers and printing the results. Obviously, these 
will perform much better after tuning their hyperparameters, but this gives you
a decent ballpark idea."""

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=50, random_state=11),
    GaussianNB()]

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [ ]:
sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")

plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')
plt.show()

sns.set_color_codes("muted")
sns.barplot(x='Log Loss', y='Classifier', data=log, color="g")

plt.xlabel('Log Loss')
plt.title('Classifier Log Loss')
plt.show()

In [ ]:
#After this choose the classifier with the best accuracy for future predictions

In [ ]:
import os
os.getcwd()